In [0]:
System_Id=dbutils.widgets.get("system_id")
Pipeline_name=dbutils.widgets.get("Pipeline_name")
Pipeline_id=dbutils.widgets.get("Pipeline_id")
Start=dbutils.widgets.get("Start")
End=dbutils.widgets.get("End")
Status=dbutils.widgets.get("Status")
Duration=dbutils.widgets.get("Duration")
Trigger_Name=dbutils.widgets.get("Trigger_Name")
Trigger_Type=dbutils.widgets.get("Trigger_Type")
Trigger_By=dbutils.widgets.get("Trigger_By")
Records_Read=dbutils.widgets.get("Records_Read")
Records_Inserted=dbutils.widgets.get("Records_Inserted")
Frequency=dbutils.widgets.get("Frequency")
Log_File=dbutils.widgets.get("Log_File")


In [0]:
System_Id=int(System_Id)
Records_Read=int(Records_Read)
Records_Inserted=int(Records_Inserted)

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, LongType, TimestampType
schema = StructType([
         StructField('Control_Id', LongType(), True),
         StructField('System_Id', LongType(), True),
         StructField('Pipeline_Name', StringType(), True),
         StructField('Pipeline_Id', StringType(), True),
         StructField('Start', StringType(), True),
         StructField('End', StringType(), True),
         StructField('Status', StringType(), True),
         StructField('Duration', StringType(), True),
         StructField('Trigger_Name', StringType(), True),
         StructField('Trigger_Type', StringType(), True),
         StructField('Triggered_By', StringType(), True),
         StructField('Records_Read', LongType(), True),
         StructField('Records_Inserted', LongType(), True),
         StructField('Frequency', StringType(), True),
         StructField('Log_File', StringType(), True),
         ])

In [0]:
control_id=0
try:
    control_id=spark.sql("select max(Control_Id) from bcp_control.tb_control").collect()[0][0]+1
except Exception as e:
    print("NO data avialable")
    control_id=1

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
df=[(control_id,System_Id,Pipeline_name,Pipeline_id,Start,End,Status,Duration,Trigger_Name,Trigger_Type,Trigger_By,Records_Read,Records_Inserted,Frequency,Log_File)]
df=spark.createDataFrame(df,schema=schema)
df = df.withColumn("Start", F.to_timestamp(F.col("Start").cast(TimestampType())))
df = df.withColumn("End", F.to_timestamp(F.col("End").cast(TimestampType())))

In [0]:
data=spark.sql("select * from bcp_control.tb_control order by Control_Id asc")
data=data.unionByName(df)
display(data)

In [0]:
def insert_data(data,TableName):
    data.write.mode("overwrite").saveAsTable(TableName)
insert_data(data,"spark_catalog.bcp_control.tb_control")

In [0]:
dbutils.notebook.exit(control_id)